In [9]:
from ast import *
def _sub_subtrees_for_vars(ast, ast_mappings):
    for node in walk(ast):
        if isinstance(node, Name) and ast_mappings.has_key(unparse(ast)):
            node = ast_mappings[unparse(ast)]
    return ast
def strip_parse(expr):
    return parse(expr)
def sub_for_func(expr, func_name, func_vars, func_expr):
    ast = strip_parse(expr)
    func_name_ast = strip_parse(func_name)
    nodes = [node for node in walk(func_name_ast.body[0])]
    print(nodes)
    if not isinstance(nodes[1], Name):
        raise ValueError('Function name is not a simple name.')
    
    func_name = func_name_ast.body[0].value.id
    print(func_name)
    func_expr_ast = strip_parse(func_expr)
    # We can strip_parse  the '*', so we special case it here.
    if func_vars == '*':
        if not hasattr(func_expr_ast, 'nodes'):
            raise ValueError("Top-level function in %s does not appear to "
                             "accept variable number of arguments. (It has no "
                             "'nodes' attribute.)" % func_expr)

        func_var_names = '*'
    else:
        func_vars_ast = [strip_parse(var).body[0].value for var in func_vars]
        for var_ast in func_vars_ast:
            if not isinstance(var_ast, Name):
                raise ValueError('Function variable is not a simple name.')
        func_var_names = [getattr(var_ast, 'id') for var_ast in func_vars_ast]
        print(func_var_names)

    ast = _sub_for_func_ast(ast, func_name, func_var_names, func_expr_ast)
#     simple = Simplify._simplify_ast(ast)
    return ast


def _sub_for_func_ast(ast, func_name, func_vars, func_expr_ast):

    print(dump(ast), func_name, func_vars, dump(func_expr_ast))
    for node in walk(ast):
    
        if isinstance(node, Call) and unparse(node.value.id) == func_name\
           and func_vars == '*':
            working_ast = copy.deepcopy(func_expr_ast)
            new_args = [_sub_for_func_ast(arg_ast, func_name, func_vars, 
                                          func_expr_ast) for arg_ast in node.value.args]
            # This subs out the arguments of the original function.
            working_ast.node.value.args = new_args
            return working_ast
        if isinstance(ast, Call) and unparse(node.value.id) == func_name\
           and len(node.value.args) == len(func_vars):
            # If our ast is the function we're looking for, we take the ast
            #  for the function expression, substitute for its arguments, and
            #  return
            working_ast = copy.deepcopy(func_expr_ast)
            mapping = {}
            for var_name, arg_ast in zip(func_vars, node.value.args):
                subbed_arg_ast = _sub_for_func_ast(arg_ast, func_name, func_vars, 
                                                   func_expr_ast)
                mapping[var_name] = subbed_arg_ast
            _sub_subtrees_for_vars(working_ast, mapping)
            return working_ast
    return ast
    

In [10]:
def test_sub_for_func():
    cases = [('f(x)', 'f', 'y', 'y+1',
              'x+1'),
             ]

    for expr, func_name, func_vars, func_expr, answer in cases:
        print
        subbed = sub_for_func(expr, func_name, func_vars,
                                           func_expr)
        assert eval(answer) == eval(subbed)

In [11]:
test_sub_for_func()


[<_ast.Expr object at 0x7f66067f6210>, <_ast.Name object at 0x7f66067fd790>, <_ast.Load object at 0x7f6606cc2950>]
f
['y']
("Module(body=[Expr(value=Call(func=Name(id='f', ctx=Load()), args=[Name(id='x', ctx=Load())], keywords=[], starargs=None, kwargs=None))])", 'f', ['y'], "Module(body=[Expr(value=BinOp(left=Name(id='y', ctx=Load()), op=Add(), right=Num(n=1)))])")


NameError: global name 'unparse' is not defined

In [5]:
import ast
flags = ast.PyCF_ONLY_AST 
tree = compile("a+b", filename='<unknown>', mode='exec',
    flags=flags, dont_inherit=True)

In [8]:
import sys
sys.executable

'/home/sudheera/dev/p4/bin/python'